In [26]:
from io import StringIO

import geopandas as gpd
import pandas as pd
import shapely
import numpy as np
import osmnx as osx
import folium

In [43]:
north = 52.39890924715826
south = 52.38917919930299
east = 13.120473448878073
west = 13.137786761544476
tags = {'highway':['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential', 
                   'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link',
                   'living_street', 'service', 'track', 'bus_guideway', 'escape', #'pedestrian',
                   'raceway', 'road', 'busway',
                   'footway', 'bridleway', 'steps', 'corridor', 'path', 'via_ferrata']}
buffer_distance = 10
example_photos_path = 'example_data/haus_l_points.json'
WORKING_CRS = 'EPSG:32632'

In [72]:
with open(example_photos_path) as f:
    photo_points = f.read()
photo_points = StringIO(photo_points)
photo_points = gpd.read_file(photo_points).to_crs(WORKING_CRS)

In [107]:
street_geoms = osx.geometries.geometries_from_bbox(north=north,south=south,east=east,west=west,tags=tags)
street_geoms = street_geoms.loc[:, ('geometry', 'highway')].reset_index('element_type')
street_geoms_buffered = street_geoms.to_crs(WORKING_CRS)
street_geoms_buffered.geometry = street_geoms_buffered.buffer(buffer_distance)
joined_streets = street_geoms_buffered.sjoin(photo_points, how="left")
photo_attributes = joined_streets.groupby('osmid')[['pollution', 'nature_effected']].mean()
street_geoms.loc[photo_attributes.index, ['pollution', 'nature_effected']] = photo_attributes
street_geoms.pollution = street_geoms.pollution.fillna(0)
street_geoms.nature_effected = street_geoms.nature_effected.fillna(0)
rgb = pd.DataFrame({'red':(street_geoms.pollution * 255 / 100), 'green':0, 'blue':0}).astype(int)
hex_number = rgb.red 
hex_number = hex_number * 256 + rgb.green
hex_number = hex_number * 256 + rgb.blue
street_geoms['stroke'] = hex_number.astype(int).apply(lambda i: f'#{i:06x}')

In [178]:
street_geoms.explore(color = street_geoms.stroke)

In [159]:
street_geoms.iloc[0].stroke

'##000000'

In [83]:
joined_streets[~joined_streets.index_right.isna()]

,element_type,geometry,highway,index_right,pollution,nature_effected
osmid,,,,,,
24605616,way,"POLYGON ((780509.264 5812730.055, 780518.294 5...",service,12.0,99.0,10.0
24605616,way,"POLYGON ((780509.264 5812730.055, 780518.294 5...",service,6.0,100.0,40.0
25672282,way,"POLYGON ((780775.913 5812869.250, 780791.852 5...",residential,2.0,42.0,30.0
27747041,way,"POLYGON ((780896.993 5812874.062, 780896.020 5...",residential,0.0,29.0,30.0
77398588,way,"POLYGON ((780957.288 5812732.020, 780958.717 5...",footway,8.0,1.0,10.0
79388180,way,"POLYGON ((780738.962 5812762.848, 780738.002 5...",service,4.0,10.0,100.0
79388258,way,"POLYGON ((780714.369 5812740.176, 780714.549 5...",service,6.0,100.0,40.0
79388407,way,"POLYGON ((780778.307 5812830.632, 780770.172 5...",service,3.0,7.0,30.0
85681876,way,"POLYGON ((781021.548 5812877.565, 781036.485 5...",footway,16.0,90.0,10.0


In [38]:
street_geoms_buffered

element_type  osmid     
way           4778401       POLYGON ((780143.898 5812202.360, 780143.586 5...
              4778620       POLYGON ((780533.017 5812583.942, 780579.296 5...
              4778621       POLYGON ((781085.628 5812656.623, 781086.609 5...
              4778622       POLYGON ((781098.339 5812657.741, 781111.020 5...
              4779619       POLYGON ((781133.593 5812624.681, 781141.959 5...
                                                  ...                        
              1149462631    POLYGON ((780530.457 5812361.556, 780531.429 5...
              1149462635    POLYGON ((780589.428 5812410.099, 780589.787 5...
              1149462636    POLYGON ((780613.809 5812396.557, 780614.342 5...
              1149462637    POLYGON ((780638.983 5812382.061, 780639.516 5...
              1149462654    POLYGON ((780561.348 5812508.786, 780560.734 5...
Length: 386, dtype: geometry

In [20]:
street_geoms_buffered.explore()

In [7]:
center = [(east + west) / 2, (north + south) / 2]
base_map = folium.Map(location=center, tiles="Stamen Terrain", zoom_start=9)
base_map

In [ ]:
for _, r in df.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['BoroName']).add_to(geo_j)
    geo_j.add_to(m)